<a href="https://colab.research.google.com/github/rudraymehra/RESS-Attrition-Prediction/blob/main/RESS_Attrition_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
##  RESS builds a transparent attrition-prediction system by combining HR datasets with
## market intelligence (reviews, salaries, hiring trends), surfaces key drivers via SHAP,
## and maps them to practical retention recommendations through a Streamlit dashboard and API.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os

BASE = "/content/drive/MyDrive/RESS-Attrition-Prediction"
folders = [
    "data/raw", "data/processed", "data/external",
    "notebooks", "src", "models", "reports/figures"
]
for f in folders:
    os.makedirs(os.path.join(BASE, f), exist_ok=True)

print("Created:", BASE)

Mounted at /content/drive
Created: /content/drive/MyDrive/RESS-Attrition-Prediction
